In [ ]:
import json
import urllib.parse
import urllib.request
import requests

In [ ]:
MAPQUEST_API_KEY = "SoCgbvqNvo1QokYCIR26DGB4DdPGp1o2"
BASE_MAPQUEST_URL = "http://www.mapquestapi.com/"

# function to build url with positions
def build_search_url_pos(start_posi, end_posi) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_posi),('to',end_posi)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)

# function to build url with lat&long
def build_search_url_latlong(start_lat:str, start_lon:str, 
                     ends_lat:str, ends_lon:str) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_lat+','+start_lon),('to',ends_lat+','+ends_lon)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)



# function to build url with start position and end lat long
def build_search_url_pos_latlong(start_posi, 
                       ends_lat:str, ends_lon:str) -> str:
    query_parameters = [('key',MAPQUEST_API_KEY),('from',start_posi),('to',ends_lat+','+ends_lon)]
    return BASE_MAPQUEST_URL +"directions/v2/route?" + urllib.parse.urlencode(query_parameters)

# function that get the web result(dictionary) of MAPQUEST
def get_result(url: str) -> dict:
    result = None
    try:
        result = urllib.request.urlopen(url)
        json_text = result.read().decode(encoding = 'utf-8')
#        print(json_text)
        return json.loads(json_text)

    finally:
        if result != None:
            result.close()
            
# get distance, units in miles
def get_distance(json_result):
    dis = json_result['route']['distance']
    return dis

#Start location is Exit 1A of I-5; end location is I-5 Exit 108

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Modeling
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/Modeling
'Copy of Geo.ipynb'   Data


In [ ]:
exits = pd.read_csv('Data/I5Exits.csv')
exits

,attributes.ExitNumber,geometry.x,geometry.y
0,1 A,-122.672895,45.621707
1,1 C,-122.667187,45.627694
2,2,-122.661702,45.644626
3,3,-122.663286,45.654278
4,4,-122.664597,45.671567
...,...,...,...
726,NaN,-122.429281,47.234529
727,NaN,-122.432021,47.236890
728,NaN,-122.205107,47.470999
729,NaN,-122.454182,47.230811


In [ ]:
# End 108, Start 1A
end_lat = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.y']
end_long = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.x']
start_lat = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.y']
start_long = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.x']

In [ ]:
url = build_search_url_latlong(str(start_lat), str(start_long), str(end_lat), str(end_long))
result = get_result(url)
get_distance(result)

107.686

# Filter gas stations within 5 mi range of I-5 1A to 108 Corridor

In [ ]:
sessionId = result['route']['sessionId']

In [ ]:
def build_corridor_search_url() -> str:
    query_parameters = [('key', MAPQUEST_API_KEY)]
    return BASE_MAPQUEST_URL +"search/v2/corridor?" + urllib.parse.urlencode(query_parameters)

def build_corridor_search_body(dataList) -> str:
    json_map = {
        "sessionId": sessionId,
        "remoteDataList": dataList,
        "options": {
            "width": '5',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)

def post_corridor_search_request(dataList):
    result = None
    url = build_corridor_search_url()
    body = build_corridor_search_body(dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()

def getRemoteDataList(df,lt,lg,name):
  df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
  remoteDataList = []
  for row in range(df.shape[0]):
    inner = {}
    r_data = df.iloc[row]
    inner['key'] = str(row)
    inner['name'] = r_data[name]
    inner['shapePoints'] = r_data['shapePoints']
    remoteDataList.append(inner)
  return remoteDataList


In [ ]:
gas_station = pd.read_csv('Data/wa_gas_stations.csv')
remoteDataList = getRemoteDataList(gas_station,'Latitude','Longitude','name')
result = post_corridor_search_request(remoteDataList)

In [ ]:
print(result['resultsCount'])

In [ ]:
filtered_gas_stations = pd.DataFrame(result['searchResults'])

In [ ]:
filtered_gas_stations.iloc[258]

# For each gas station, find closest exit

In [ ]:

def build_radius_search_url() -> str:
    query_parameters = [('key', MAPQUEST_API_KEY)]
    return BASE_MAPQUEST_URL +"search/v2/radius?" + urllib.parse.urlencode(query_parameters)

def build_radius_search_body(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '5',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)

def post_radius_search_request(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()


In [ ]:
exits = pd.read_csv('Data/I5Exits.csv')
exitDataList = getRemoteDataList(exits,'geometry.y','geometry.x','attributes.ExitNumber')
origin = filtered_gas_stations.iloc[0]['shapePoints']
exit_result = post_radius_search_request(origin, exitDataList)
exit_result['resultsCount']

In [ ]:
def get_closest_exit(origin):
    exit_result = post_radius_search_request(origin, exitDataList)
    filtered_exits = pd.DataFrame(exit_result['searchResults'])
    closest_exit = filtered_exits.iloc[filtered_exits['distance'].argmin()]
    return closest_exit['distance'], closest_exit['name'], closest_exit['shapePoints'] 

In [ ]:
dist_to_exit = []
for gas_r in range(filtered_gas_stations.shape[0]):
  gas_data = filtered_gas_stations.iloc[gas_r]
  dist, exit_n, exit_p = get_closest_exit(gas_data['shapePoints'])
  dist_to_exit.append([gas_data['key'],gas_data['shapePoints'][0],gas_data['shapePoints'][1],exit_n,exit_p[0],exit_p[1],dist])
dist_to_exit = pd.DataFrame(dist_to_exit)
dist_to_exit.columns = ['gas_key','gas_lat','gas_long','exit_name','exit_lat','exit_long','distance']

In [ ]:
dist_to_exit.to_csv('Data/dist_to_I5exit.csv')

# For each gas station, find number of EV stations(>= level 2) within range 20 mile

In [ ]:


def getRemoteEvDataList(df,lt,lg,name):

    df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
    remoteDataList = []
    for row in range(df.shape[0]):
        inner = {}
        r_data = df.iloc[row]
        inner['key'] = str(row)
        inner['name'] = r_data['Station_Name']
        inner['shapePoints'] = r_data['shapePoints']
        remoteDataList.append(inner)
    return remoteDataList


# within 2 miles
def build_radius_search_body_2mile(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '2',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)


def post_radius_search_ev_request_2(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body_2mile(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()

# within 5 miles
def build_radius_search_body_5mile(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '5',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)


def post_radius_search_ev_request_5(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body_5mile(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()


# within 10 miles
def build_radius_search_body_10mile(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '10',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)


def post_radius_search_ev_request_10(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body_10mile(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()


# within 20 miles
def build_radius_search_body_20mile(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '20',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)


def post_radius_search_ev_request_20(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body_20mile(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()

# within 50 miles
def build_radius_search_body_50mile(origin, dataList) -> str:
    json_map = {
        "origin": {
            "latLng": { 
                "lat": origin[0], 
                "lng": origin[1]
            }
        },
        "remoteDataList": dataList,
        "options": {
            "radius": '50',
            "units": 'm',
            "maxMatches": 500
        }
    }
    return json.dumps(json_map)


def post_radius_search_ev_request_50(origin, dataList):
    result = None
    url = build_radius_search_url()
    body = build_radius_search_body_50mile(origin, dataList)
    try:
        result = requests.post(url, body)
        return result.json()

    finally:
        if result != None:
            result.close()

In [ ]:
ev_station = pd.read_csv('Data/alt_fuel_stations(Sep_25_2022).csv')
remoteDataListEv = getRemoteEvDataList(ev_station,'Latitude','Longitude','Station_Name')

In [ ]:
# ev_result_2 = post_radius_search_ev_request_2(origin, remoteDataListEv)

In [ ]:
def get_number_ev_2(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_2 = post_radius_search_ev_request_2(origin, evDataList)
    ev_2 = ev_result_2['resultsCount']
    return ev_2

def get_number_ev_5(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_5 = post_radius_search_ev_request_5(origin, evDataList)
    ev_5 = ev_result_5['resultsCount']
    return ev_5 
def get_number_ev_10(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_10 = post_radius_search_ev_request_10(origin, evDataList)
    ev_10 = ev_result_10['resultsCount']
    return ev_10 

def get_number_ev_20(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_20 = post_radius_search_ev_request_20(origin, evDataList)
    ev_20 = ev_result_20['resultsCount']
    return ev_20 

def get_number_ev_50(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_50 = post_radius_search_ev_request_50(origin, evDataList)
    ev_50 = ev_result_50['resultsCount']
    return ev_50 

In [ ]:
def get_number_ev(origin, evDataList):
    # find EV within 2 miles of gas stations
    ev_result_2 = post_radius_search_ev_request_2(origin, evDataList)
    ev_2 = ev_result_2['resultsCount']
    #print('2')

    # find EV within 5 miles of gas stations
    ev_result_5 = post_radius_search_ev_request_5(origin, evDataList)
    ev_5 = ev_result_5['resultsCount']
    #print('5')

    # find EV within 10 miles of gas stations
    ev_result_10 = post_radius_search_ev_request_10(origin, evDataList)
    ev_10 = ev_result_10['resultsCount']
    #print('10')

    # find EV within 20 miles of gas stations
    ev_result_20 = post_radius_search_ev_request_20(origin, evDataList)
    ev_20 = ev_result_20['resultsCount']
    #print('20')

    # find EV within 50 miles of gas stations
    #ev_result_50 = post_radius_search_ev_request_50(origin, evDataList)
    #ev_50 = ev_result_50['resultsCount']
    #print('50')

    return ev_2, ev_5, ev_10, ev_20

# Single run

In [ ]:
gas_data_set = pd.DataFrame(data=remoteDataList)
num_of_ev = []
for gas_r in range(gas_data_set.shape[0]):
  gas_data = gas_data_set.iloc[gas_r]
  
  #gas_lat_long = gas_data['shapePoints'] 
  #print(gas_lat_long)
  #print([gas_lat_long[0],gas_lat_long[1]])
  num_ev_2 = get_number_ev_2([gas_data['shapePoints'][0],gas_data['shapePoints'][1]],remoteDataListEv)
  num_of_ev.append([gas_data['key'], gas_data['shapePoints'][0], gas_data['shapePoints'][1], num_ev_2])
gas_data_set_df = pd.DataFrame(num_of_ev.append)

# multiple run

In [ ]:
# gas_data_set = pd.DataFrame(data=remoteDataList)
# num_of_ev = []
# for gas_r in range(gas_data_set.shape[0]):
#   gas_data = gas_data_set.iloc[gas_r]
  
#   #gas_lat_long = gas_data['shapePoints'] 
#   #print(gas_lat_long)
#   #print([gas_lat_long[0],gas_lat_long[1]])
#   num_ev_2, num_ev_5, num_ev_10, num_ev_20 = get_number_ev([gas_data['shapePoints'][0],gas_data['shapePoints'][1]],remoteDataListEv)
#   num_of_ev.append([gas_data['key'], gas_data['shapePoints'][0], gas_data['shapePoints'][1], num_ev_2, num_ev_5, num_ev_10, num_ev_20])


ValueError: ignored

In [ ]:
gas_data_set_df = pd.DataFrame(num_of_ev)
gas_data_set_df.columns = ['gas_key','gas_lat','gas_long','num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas', 'num_EV_in_10_miles_of_gas','num_EV_in_20_miles_of_gas']
gas_data_set_df

,gas_key,gas_lat,gas_long,num_EV_in_2_miles_of_gas,num_EV_in_5_miles_of_gas,num_EV_in_10_miles_of_gas,num_EV_in_20_miles_of_gas
0,0,48.393760,-122.329806,3,14,16,36
1,1,47.191712,-120.916225,2,4,4,4
2,2,47.238704,-122.357577,2,49,125,241
3,3,48.435227,-122.291324,5,14,16,35
4,4,47.839608,-120.014523,2,4,7,11
...,...,...,...,...,...,...,...
2206,2206,48.198460,-122.127847,2,5,10,46
2207,2207,48.556192,-122.344545,0,1,14,55
2208,2208,48.747436,-122.475199,17,27,27,46
2209,2209,48.755411,-122.463930,16,27,27,45


In [ ]:
gas_data_set_df.to_csv('Data/gas_station_w_ev_station_cnt.csv')

# For each gas station, find distance between closest existing ev charging stations

In [ ]:
 origin = filtered_gas_stations.iloc[0]['shapePoints']
 ev_result = post_radius_search_ev_request(origin, remoteDataListEv)
 ev_result

In [ ]:
def get_closest_ev_50(origin, evDataList):
    ev_result = post_radius_search_ev_request_50(origin, evDataList)
    filtered_ev = pd.DataFrame(ev_result['searchResults'])
    closest_ev = filtered_ev.iloc[filtered_ev['distance'].argmin()]
    return ev_result['resultsCount'], closest_ev['distance'], closest_ev['name'], closest_ev['shapePoints'] 


In [ ]:
data_set = pd.read_csv('Data/gas_station_w_ev_station_cnt.csv')
closest_ev_to_gas = []
for gas_r in range(data_set.shape[0]):
  gas_data = data_set.iloc[gas_r]
  ev_50_count, dist_ev, ev_n, ev_p = get_closest_ev_50([gas_data['gas_lat'],gas_data['gas_long']],remoteDataListEv)
  closest_ev_to_gas.append([ev_50_count, ev_n,ev_p[0],ev_p[1],dist_ev])
dist_to_ev = pd.DataFrame(closest_ev_to_gas)
dist_to_ev.columns = ['num_EV_in_50_miles_of_gas','Closest_EV_Station_name','Closest_EV_Station_lat','Closest_EV_Station_long','Closest_distance']

In [ ]:
len(dist_to_ev)

2211

In [ ]:
new_df = pd.concat([data_set, dist_to_ev], axis=1).iloc[:,1:]

In [ ]:
new_df

,gas_key,gas_lat,gas_long,num_EV_in_2_miles_of_gas,num_EV_in_5_miles_of_gas,num_EV_in_10_miles_of_gas,num_EV_in_20_miles_of_gas,num_EV_in_50_miles_of_gas,Closest_EV_Station_name,Closest_EV_Station_lat,Closest_EV_Station_long,Closest_distance
0,0,48.393760,-122.329806,3,14,16,36,291,IBEW Local 191,48.399012,-122.333583,0.4026
1,1,47.191712,-120.916225,2,4,4,4,67,Cle Elum,47.188833,-120.903139,0.6466
2,2,47.238704,-122.357577,2,49,125,241,500,BJ's Bingo & Gaming,47.243940,-122.369825,0.6797
3,3,48.435227,-122.291324,5,14,16,35,233,Skagit Valley College,48.436733,-122.312699,0.9865
4,4,47.839608,-120.014523,2,4,7,11,49,City of Chelan,47.840838,-120.018402,0.1992
...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2206,48.198460,-122.127847,2,5,10,46,500,Stillaguamish Conference Room,48.201607,-122.128261,0.2185
2207,2207,48.556192,-122.344545,0,1,14,55,141,Fairfield Inn & Suites Burlington - Tesla Supe...,48.509743,-122.338681,3.2241
2208,2208,48.747436,-122.475199,17,27,27,46,98,CFC Holly ST Lot,48.747280,-122.476220,0.0478
2209,2209,48.755411,-122.463930,16,27,27,45,96,Bellingham Nissan,48.755607,-122.454132,0.4470


In [ ]:
new_df.to_csv('Data/gas_station_w_dist_to_exit_I5.csv')